In [1]:
#encoding=utf-8
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [2]:
# path = '/home/kai/data/jiangning/Chatbot_1/Chatbot1.0/data/'
path = '../../data/CutDebt/'
data = pd.read_csv(path + 'cleaned_mock_up_data.csv', encoding='utf8')
print(data.shape)
data.head(10)

(3205, 2)


,label,split_text
0,1,多 少
1,2,我 必须 检查
2,2,我 是 他 的 兄弟
3,2,你 是 机器人 吗 ？
4,0,最好 的
5,0,非常 好 我 有 足够 的 钱 返回 这部分 。
6,0,不错
7,2,灯光 说 我 很 忙
8,1,期
9,1,不 它 太 高 了 。


In [3]:
data.split_text.iloc[89]

'我 很 忙'

In [4]:
phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

print('fitting phrase')
phrase_vectorizer.fit(data.split_text)

print('transform phrase')
phrase = phrase_vectorizer.transform(data.split_text)

phrase

fitting phrase
transform phrase


<3205x7975 sparse matrix of type '<class 'numpy.float64'>'
	with 26922 stored elements in Compressed Sparse Row format>

# SVC

In [5]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC

l_svc = LinearSVC()
clf = CalibratedClassifierCV(l_svc) 
clf.fit(phrase, data.label)
print(clf.score(phrase, data.label))

0.990639625585


In [6]:
print(clf.predict(phrase))

[1 2 2 ..., 0 2 2]


# Logistic

In [7]:
from sklearn.linear_model import LogisticRegression

In [8]:
log_r = LogisticRegression()
log_r.fit(phrase, data.label)
print(log_r.score(phrase, data.label))

0.973478939158


In [9]:
print(log_r.predict(phrase))

[1 2 2 ..., 0 2 2]


# LightGBM

In [10]:
import lightgbm as lgb

In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data.label.values)
onelabels = le.transform(data.label.values)

In [12]:
onelabels

array([1, 2, 2, ..., 0, 2, 2])

In [13]:
from sklearn.preprocessing import MultiLabelBinarizer
multicoder = MultiLabelBinarizer()
lables = multicoder.fit_transform([data.label.values])

In [14]:
lables

array([[1, 1, 1]])

In [15]:
data.label.values

array([1, 2, 2, ..., 0, 2, 2])

In [16]:
params = {
    'learning_rate': 0.2,
    'num_iterations':1000,
    'application': 'multiclassova',
    'num_class': 3,
    'num_leaves': 31,
    'verbosity': -1,
    'metric': 'multi_error',
    'data_random_seed': 2,
#     'bagging_fraction': 0.8,
#     'feature_fraction': 0.6,
    'nthread': 4,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'early_stopping_round':200
} 

# lgbm_train = lgb.Dataset(phrase, data.label)
lgbm_train = lgb.Dataset(phrase, onelabels)
lgbm_val = lgb.Dataset(phrase, onelabels)
lgbm_model = lgb.train(params,lgbm_train,valid_sets=lgbm_val, verbose_eval=5)

Training until validation scores don't improve for 200 rounds.
[5]	valid_0's multi_error: 0.219969
[10]	valid_0's multi_error: 0.201872
[15]	valid_0's multi_error: 0.185023
[20]	valid_0's multi_error: 0.165679
[25]	valid_0's multi_error: 0.157878
[30]	valid_0's multi_error: 0.145086
[35]	valid_0's multi_error: 0.137598
[40]	valid_0's multi_error: 0.132605
[45]	valid_0's multi_error: 0.127613
[50]	valid_0's multi_error: 0.125429
[55]	valid_0's multi_error: 0.122309


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[60]	valid_0's multi_error: 0.120125
[65]	valid_0's multi_error: 0.117005
[70]	valid_0's multi_error: 0.114821
[75]	valid_0's multi_error: 0.113573
[80]	valid_0's multi_error: 0.1117
[85]	valid_0's multi_error: 0.11014
[90]	valid_0's multi_error: 0.108892
[95]	valid_0's multi_error: 0.107956
[100]	valid_0's multi_error: 0.10546
[105]	valid_0's multi_error: 0.104836
[110]	valid_0's multi_error: 0.104524
[115]	valid_0's multi_error: 0.1039
[120]	valid_0's multi_error: 0.102964
[125]	valid_0's multi_error: 0.103276
[130]	valid_0's multi_error: 0.102028
[135]	valid_0's multi_error: 0.102028
[140]	valid_0's multi_error: 0.10078
[145]	valid_0's multi_error: 0.100156
[150]	valid_0's multi_error: 0.100156
[155]	valid_0's multi_error: 0.100156
[160]	valid_0's multi_error: 0.100156
[165]	valid_0's multi_error: 0.099844
[170]	valid_0's multi_error: 0.099532
[175]	valid_0's multi_error: 0.099844
[180]	valid_0's multi_error: 0.098908
[185]	valid_0's multi_error: 0.0982839
[190]	valid_0's multi_erro

In [17]:
print(lgbm_model.predict(phrase))

[[ 0.00752393  0.94258957  0.01539697]
 [ 0.05998482  0.05655368  0.89233531]
 [ 0.00309932  0.00291562  0.99514908]
 ..., 
 [ 0.81835117  0.05188742  0.15813258]
 [ 0.11976506  0.19522802  0.55813174]
 [ 0.00470172  0.00748868  0.99785889]]


# Save

In [18]:
import pickle

# save tfidf
pickle.dump(phrase_vectorizer, open("../../savedModel/CutDebt/tfidf.pickle", "wb"))
# pickle.dump(train_comment_features, open("train_comment_features.pickle", "wb"))
# pickle.dump(test_comment_features, open("test_comment_features.pickle", "wb"))

# save linear svc
pickle.dump(clf, open("../../savedModel/CutDebt/LinearSVC.pickle", "wb"))
# save logistic
pickle.dump(log_r, open("../../savedModel/CutDebt/Logistic.pickle", "wb"))
# save lightGBM
pickle.dump(lgbm_model, open("../../savedModel/CutDebt/Lgbm.pickle", "wb"))

In [29]:
import jieba
sentence = '那好吧'
sentence = jieba.cut(sentence, cut_all = False)
sentence = ' '.join(sentence)
test = phrase_vectorizer.transform([sentence])

In [30]:
clf.predict_proba(test) # linear svc

array([[ 0.78189525,  0.05591929,  0.16218545]])

In [31]:
log_r.predict_proba(test) # logistic

array([[ 0.60577253,  0.17736826,  0.2168592 ]])

In [32]:
lgbm_model.predict(test) # light gbm

array([[ 0.64207298,  0.10682254,  0.27922329]])

# Labeling

In [23]:
# basic logic: find the max probability of 3 models, if it is larger than threshold, return the corresponding label, otherwise, return 2 (others).
result = np.vstack((clf.predict_proba(test),log_r.predict_proba(test),lgbm_model.predict(test)))
pos = np.where(result == np.max(result))

threshold = 0.7
if np.max(result)<threshold:
    label = 2
else:
    label = pos[1]
    label = label[0]
    
print('label=',label)
print('prob=',np.max(result))

label= 1
prob= 0.983986058554


# Save the whole thing

In [24]:
# import jieba
# import numpy as np

# class CutDebt:
    
#     def __init__(self, **model):
#         """
#         suggested parameters:
#         svc, logistic, lightgbm, jieba_path,tfidf
#         """
#         self._load_model(**model)
        
#     def _load_model(self,**model):
#         self.svc = model.get('svc')
#         self.logistic = model.get('logistic')
#         self.lightgbm = model.get('lightgbm')
#         self.tfidf = model.get('tfidf')
#         # load jieba
#         jieba_path = model.get('jieba_path')
#         if jieba_path is not None:
#             jieba.load_userdict(jieba_path)
        
        
#     def classify(self, sentence):
#         sentence = jieba.cut(sentence, cut_all = False)
#         sentence = ' '.join(sentence)
#         matrix = self.tfidf.transform([sentence])
#         result = np.vstack((self.svc.predict_proba(matrix),
#                             self.logistic.predict_proba(matrix),
#                             self.lightgbm.predict(matrix)))
#         max_pred = np.max(result, axis=0)
#         max_arg = np.argmax(max_pred)
#         threshold = 0.6
#         if np.max(max_pred)<threshold:
#             label = 2
#         else:
#             label = max_arg
#         return (label, np.max(max_pred))
     
        

In [25]:
%load_ext autoreload
%autoreload 2

In [26]:
from CutDebt_py import CutDebt
cutd = CutDebt(svc=clf, logistic=log_r, lightgbm=lgbm_model, tfidf=phrase_vectorizer, jieba_path='../WordCut/userdict.txt')

pickle.dump(cutd, open("../../savedModel/CutDebt/CutDebt.pickle", "wb"))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.432 seconds.
Prefix dict has been built succesfully.
